This notebook illustrates incompatibility issue discussed in the paper https://openreview.net/forum?id=7MYznm5Kp2.

We create a DGP (a Bayesian network) w.r.t. ZI MCAR graph in Figure 2(a), and obtain the ground-truth $p(W|R)$. We then sample $100000$ data points $(W_i, X_i)$ from this DGP, and estimate $\hat{p}(W,X)$ by counting (MLE for categorical data). Finally, we calculate $p(R,X)$ using the Kuroki-Pearl matrix inversion equation. The estimated $p(R,X)$ has negative elements, rendering it invalid.
```
[ 0.51789683 -0.08300896]
[ 0.05589317  0.50921896]
```

In [15]:
import numpy as np
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
from pgmpy.factors.discrete.CPD import TabularCPD

def add_ZI_consistency_edge(net):
    card_R = net.get_cardinality("R")
    card_X1 = net.get_cardinality("X1")
    """
    cpd table for p(x | x(1), r)
    ---
    R=   |     0     |     1     |
    X1=  |   0 |   1 |   0 |   1 |
    ---
    X=0  | 1.0 | 1.0 | 1.0 | 0.0 |
    X=1  | 0.0 | 0.0 | 0.0 | 1.0 |
    ---
    """
    cpd_X = TabularCPD(
        "X",
        2,
        [[1.0, 1.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]],
        evidence=["R", "X1"],
        evidence_card=[card_R, card_X1],
    )
    net.add_cpds(cpd_X)
    return net


def odd_ratio(p, c=None):
    """
    Input:
        If c=None, p is 2x2 matrix with p[i,j] = p(a=i|b=j).
        If c=0,1, p is 2x2x2 matrix with p[i,j,k] = p(a=i|b=j,c=k).
    Output: Odd-ratio p(a=1|b=1,c) / p(a=0|b=1,c) * p(a=0|b=0,c) / p(a=1|b=0,c)
    """
    if c == None:
        return p[1, 1] / p[0, 1] * p[0, 0] / p[1, 0]
    elif isinstance(c, int):
        return p[1, 1, c] / p[0, 1, c] * p[0, 0, c] / p[1, 0, c]


class CaseMCAR:
    def __init__(self, cards):
        """
        Create the graph X(1) -> X <- R -> W
        """
        self.vertices = ["X1", "X", "R", "W"]
        self.cards = cards
        self.edges = [("X1", "X"), ("R", "X"), ("R", "W")]
        self.net = BayesianNetwork(self.edges)
        # self.net.to_daft(node_pos={'X': (0,0), 'X1': (0,1), 'R': (1,0), 'W': (2,0)}).render()
        self.get_random_cpds()
        # If |x| < epsilon, then x is consider "zero" (small noise)
        self.eps = 1e-15
        self.eps2 = 1e-12
        self.lb, self.ub = None, None
        self.num_lb, self.num_ub = None, None

    def get_random_cpds(self):
        """
        Get a random data-generation process in the model, i.e., satisfying 3 conditions
        1. random cpds
        2. applying the ZI consistency
        3. W must associate to R
        """
        while True:
            self.net.get_random_cpds(n_states=self.cards, inplace=True)
            self.net = add_ZI_consistency_edge(self.net)
            infer = VariableElimination(self.net)
            self.p_WR = infer.query(["W", "R"]).values
            self.p_WX = infer.query(["W", "X"]).values
            # Proxy assumption: W must assoc with X, e.g., OR cannot be 1
            # Marginal assumption: W must assoc with X
            if (odd_ratio(self.p_WR, c=None) != 1) and (
                odd_ratio(self.p_WX, c=None) != 1
            ):
                break

        # Compute the conditional distributions p(w|r) and p(w|x)
        self.p_RX = infer.query(["R", "X"]).values
        self.p_W_R = np.zeros_like(self.p_WR)
        for r in range(self.cards["R"]):
            self.p_W_R[:, r] = self.p_WR[:, r] / np.sum(self.p_WR[:, r])


np.random.seed(81)

# Create the DGP
print(f"MCAR case. Graph: X(1) -> X <- R -> W")
cards = {"X1": 2, "X": 2, "R": 2, "W": 2}
case_mcar = CaseMCAR(cards)
case_mcar.get_random_cpds()

# True pieces
print("\n")
print("True p(W,X):\n", case_mcar.p_WX)
print("True p(W|R):\n", case_mcar.p_W_R)
print("True p(R,X):\n", case_mcar.p_RX)

# Calculate p(R,X) using Kuroki-Pearl method, use true p(W,X) and true p(W|R)
hat_p_RX = np.matmul(np.linalg.inv(case_mcar.p_W_R), case_mcar.p_WX)
print("Computed p(R,X) via matrix inversion using true p(W,X) and true p(W|R):\n", hat_p_RX)
print("Element-wise difference to true p(R,X):\n", np.abs(hat_p_RX - case_mcar.p_RX))

# Sample data from the DGP
print("\n")
samples = case_mcar.net.simulate(100000)

# Estimate p(W,X) from data using plug-in estimator
hat_p_WX = np.zeros((2, 2))
hat_p_WX[0,0] = np.mean((samples.W == 0) * (samples.X == 0))
hat_p_WX[1,0] = np.mean((samples.W == 1) * (samples.X == 0))
hat_p_WX[0,1] = np.mean((samples.W == 0) * (samples.X == 1))
hat_p_WX[1,1] = np.mean((samples.W == 1) * (samples.X == 1))
print("Estimated p(W,X):\n", hat_p_WX)

# Calculate p(R,X) using Kuroki-Pearl method, use estimated p(W,X) and true p(W|R)
hat_p_RX = np.matmul(np.linalg.inv(case_mcar.p_W_R), hat_p_WX)
print("Computed p(R,X) via matrix inversion using estimated p(W,X) and true p(W|R):\n", hat_p_RX)
print("Element-wise difference to true p(R,X):\n", np.abs(hat_p_RX - case_mcar.p_RX))

MCAR case. Graph: X(1) -> X <- R -> W


True p(W,X):
 [[0.42643891 0.31215362]
 [0.14620603 0.11520144]]
True p(W|R):
 [[0.74919143 0.73043156]
 [0.25080857 0.26956844]]
True p(R,X):
 [[0.43502295 0.        ]
 [0.13762199 0.42735506]]
Computed p(R,X) via matrix inversion using true p(W,X) and true p(W|R):
 [[ 4.35022949e-01 -1.81411279e-16]
 [ 1.37621992e-01  4.27355059e-01]]
Element-wise difference to true p(R,X):
 [[1.33226763e-15 1.81411279e-16]
 [1.38777878e-16 4.99600361e-16]]




  0%|          | 0/4 [00:00<?, ?it/s]

Estimated p(W,X):
 [[0.42883 0.30976]
 [0.14496 0.11645]]
Computed p(R,X) via matrix inversion using estimated p(W,X) and true p(W|R):
 [[ 0.51789683 -0.08300896]
 [ 0.05589317  0.50921896]]
Element-wise difference to true p(R,X):
 [[0.08287388 0.08300896]
 [0.08172882 0.0818639 ]]
